# Machine learning - hospital readmission

## (I) Preprocessing

In [2]:
import numpy as np
import pandas as pd
import os
os.chdir('/Users/ljt/Documents/Madison/jupyter/DataChat/dataset_diabetes/')
#os.getcwd()

In [ ]:
data = pd.read_csv('diabetic_data.csv')

In [ ]:
print(data.shape)
#print(data.info())
print(data.describe())

In [ ]:
df = data.copy(deep=True)

In [ ]:
#Print all the variables with missing values.
for col in df.columns:
    if df[col].dtype == object:
        if df[col][df[col] == '?'].count():
            print(col,df[col][df[col] == '?'].count())
print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count())

In [ ]:
df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)#drop columns

In [ ]:
drop_Idx = set(df[(df['diag_1'] == '?') & (df['diag_2'] == '?') & (df['diag_3'] == '?')].index)
drop_Idx = drop_Idx.union(set(df['gender'][df['gender'] == 'Unknown/Invalid'].index))
drop_Idx = drop_Idx.union(set(df[df['discharge_disposition_id'] == 11].index))
new_Idx = list(set(df.index) - set(drop_Idx))
df = df.iloc[new_Idx]

In [ ]:
df = df.drop(['citoglipton', 'examide'], axis = 1)#same value for all observations

In [ ]:
df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']

In [ ]:
# re-encoding admission type, discharge type and admission source into fewer categories

df['admission_type_id'] = df['admission_type_id'].replace(2,1)
df['admission_type_id'] = df['admission_type_id'].replace(7,1)
df['admission_type_id'] = df['admission_type_id'].replace(6,5)
df['admission_type_id'] = df['admission_type_id'].replace(8,5)

df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(6,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(8,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(9,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(13,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(3,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(4,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(5,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(14,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(22,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(23,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(24,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(12,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(15,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(16,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(17,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(25,18)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(26,18)

df['admission_source_id'] = df['admission_source_id'].replace(2,1)
df['admission_source_id'] = df['admission_source_id'].replace(3,1)
df['admission_source_id'] = df['admission_source_id'].replace(5,4)
df['admission_source_id'] = df['admission_source_id'].replace(6,4)
df['admission_source_id'] = df['admission_source_id'].replace(10,4)
df['admission_source_id'] = df['admission_source_id'].replace(22,4)
df['admission_source_id'] = df['admission_source_id'].replace(25,4)
df['admission_source_id'] = df['admission_source_id'].replace(15,9)
df['admission_source_id'] = df['admission_source_id'].replace(17,9)
df['admission_source_id'] = df['admission_source_id'].replace(20,9)
df['admission_source_id'] = df['admission_source_id'].replace(21,9)
df['admission_source_id'] = df['admission_source_id'].replace(13,11)
df['admission_source_id'] = df['admission_source_id'].replace(14,11)

In [ ]:
#Print all the variables with missing values.
for col in df.columns:
    if df[col].dtype == object:
        if df[col][df[col] == '?'].count():
            print(col,df[col][df[col] == '?'].count())
print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count())

In [ ]:
keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
for col in keys:
    colname = str(col) + 'temp'
    df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
df['numchange'] = 0
for col in keys:
    colname = str(col) + 'temp'
    df['numchange'] = df['numchange'] + df[colname]
    del df[colname]
df['numchange'].value_counts()

In [ ]:
df.shape

In [ ]:
df['nummed'] = 0
for col in keys:
    df['nummed'] = df['nummed'] + df[col].apply(lambda x: 0 if (x == 'No') else 1)
df['nummed'].value_counts()

In [ ]:
# Creating additional columns for diagnosis
df['level1_diag1'] = df['diag_1']
df['level2_diag1'] = df['diag_1']
df['level1_diag2'] = df['diag_2']
df['level2_diag2'] = df['diag_2']
df['level1_diag3'] = df['diag_3']
df['level2_diag3'] = df['diag_3']
df.loc[df['diag_1'].str.contains('V'), ['level1_diag1', 'level2_diag1']] = 0
df.loc[df['diag_1'].str.contains('E'), ['level1_diag1', 'level2_diag1']] = 0
df.loc[df['diag_2'].str.contains('V'), ['level1_diag2', 'level2_diag2']] = 0
df.loc[df['diag_2'].str.contains('E'), ['level1_diag2', 'level2_diag2']] = 0
df.loc[df['diag_3'].str.contains('V'), ['level1_diag3', 'level2_diag3']] = 0
df.loc[df['diag_3'].str.contains('E'), ['level1_diag3', 'level2_diag3']] = 0
df['level1_diag1'] = df['level1_diag1'].replace('?', -1)
df['level2_diag1'] = df['level2_diag1'].replace('?', -1)
df['level1_diag2'] = df['level1_diag2'].replace('?', -1)
df['level2_diag2'] = df['level2_diag2'].replace('?', -1)
df['level1_diag3'] = df['level1_diag3'].replace('?', -1)
df['level2_diag3'] = df['level2_diag3'].replace('?', -1)

In [ ]:
df['level1_diag1'] = df['level1_diag1'].astype(float)
df['level2_diag1'] = df['level2_diag1'].astype(float)
df['level1_diag2'] = df['level1_diag2'].astype(float)
df['level2_diag2'] = df['level2_diag2'].astype(float)
df['level1_diag3'] = df['level1_diag3'].astype(float)
df['level2_diag3'] = df['level2_diag3'].astype(float)

In [ ]:
for index, row in df.iterrows():
    if (row['level1_diag1'] >= 390 and row['level1_diag1'] < 460) or (np.floor(row['level1_diag1']) == 785):
        df.loc[index, 'level1_diag1'] = 1
    elif (row['level1_diag1'] >= 460 and row['level1_diag1'] < 520) or (np.floor(row['level1_diag1']) == 786):
        df.loc[index, 'level1_diag1'] = 2
    elif (row['level1_diag1'] >= 520 and row['level1_diag1'] < 580) or (np.floor(row['level1_diag1']) == 787):
        df.loc[index, 'level1_diag1'] = 3
    elif (np.floor(row['level1_diag1']) == 250):
        df.loc[index, 'level1_diag1'] = 4
    elif (row['level1_diag1'] >= 800 and row['level1_diag1'] < 1000):
        df.loc[index, 'level1_diag1'] = 5
    elif (row['level1_diag1'] >= 710 and row['level1_diag1'] < 740):
        df.loc[index, 'level1_diag1'] = 6
    elif (row['level1_diag1'] >= 580 and row['level1_diag1'] < 630) or (np.floor(row['level1_diag1']) == 788):
        df.loc[index, 'level1_diag1'] = 7
    elif (row['level1_diag1'] >= 140 and row['level1_diag1'] < 240):
        df.loc[index, 'level1_diag1'] = 8
    else:
        df.loc[index, 'level1_diag1'] = 0
        
    if (row['level1_diag2'] >= 390 and row['level1_diag2'] < 460) or (np.floor(row['level1_diag2']) == 785):
        df.loc[index, 'level1_diag2'] = 1
    elif (row['level1_diag2'] >= 460 and row['level1_diag2'] < 520) or (np.floor(row['level1_diag2']) == 786):
        df.loc[index, 'level1_diag2'] = 2
    elif (row['level1_diag2'] >= 520 and row['level1_diag2'] < 580) or (np.floor(row['level1_diag2']) == 787):
        df.loc[index, 'level1_diag2'] = 3
    elif (np.floor(row['level1_diag2']) == 250):
        df.loc[index, 'level1_diag2'] = 4
    elif (row['level1_diag2'] >= 800 and row['level1_diag2'] < 1000):
        df.loc[index, 'level1_diag2'] = 5
    elif (row['level1_diag2'] >= 710 and row['level1_diag2'] < 740):
        df.loc[index, 'level1_diag2'] = 6
    elif (row['level1_diag2'] >= 580 and row['level1_diag2'] < 630) or (np.floor(row['level1_diag2']) == 788):
        df.loc[index, 'level1_diag2'] = 7
    elif (row['level1_diag2'] >= 140 and row['level1_diag2'] < 240):
        df.loc[index, 'level1_diag2'] = 8
    else:
        df.loc[index, 'level1_diag2'] = 0
    
    if (row['level1_diag3'] >= 390 and row['level1_diag3'] < 460) or (np.floor(row['level1_diag3']) == 785):
        df.loc[index, 'level1_diag3'] = 1
    elif (row['level1_diag3'] >= 460 and row['level1_diag3'] < 520) or (np.floor(row['level1_diag3']) == 786):
        df.loc[index, 'level1_diag3'] = 2
    elif (row['level1_diag3'] >= 520 and row['level1_diag3'] < 580) or (np.floor(row['level1_diag3']) == 787):
        df.loc[index, 'level1_diag3'] = 3
    elif (np.floor(row['level1_diag3']) == 250):
        df.loc[index, 'level1_diag3'] = 4
    elif (row['level1_diag3'] >= 800 and row['level1_diag3'] < 1000):
        df.loc[index, 'level1_diag3'] = 5
    elif (row['level1_diag3'] >= 710 and row['level1_diag3'] < 740):
        df.loc[index, 'level1_diag3'] = 6
    elif (row['level1_diag3'] >= 580 and row['level1_diag3'] < 630) or (np.floor(row['level1_diag3']) == 788):
        df.loc[index, 'level1_diag3'] = 7
    elif (row['level1_diag3'] >= 140 and row['level1_diag3'] < 240):
        df.loc[index, 'level1_diag3'] = 8
    else:
        df.loc[index, 'level1_diag3'] = 0

In [ ]:
for index, row in df.iterrows():
    if (row['level2_diag1'] >= 390 and row['level2_diag1'] < 399):
        df.loc[index, 'level2_diag1'] = 1
    elif (row['level2_diag1'] >= 401 and row['level2_diag1'] < 415):
        df.loc[index, 'level2_diag1'] = 2
    elif (row['level2_diag1'] >= 415 and row['level2_diag1'] < 460):
        df.loc[index, 'level2_diag1'] = 3
    elif (np.floor(row['level2_diag1']) == 785):
        df.loc[index, 'level2_diag1'] = 4
    elif (row['level2_diag1'] >= 460 and row['level2_diag1'] < 489):
        df.loc[index, 'level2_diag1'] = 5
    elif (row['level2_diag1'] >= 490 and row['level2_diag1'] < 497):
        df.loc[index, 'level2_diag1'] = 6
    elif (row['level2_diag1'] >= 500 and row['level2_diag1'] < 520):
        df.loc[index, 'level2_diag1'] = 7
    elif (np.floor(row['level2_diag1']) == 786):
        df.loc[index, 'level2_diag1'] = 8
    elif (row['level2_diag1'] >= 520 and row['level2_diag1'] < 530):
        df.loc[index, 'level2_diag1'] = 9
    elif (row['level2_diag1'] >= 530 and row['level2_diag1'] < 544):
        df.loc[index, 'level2_diag1'] = 10
    elif (row['level2_diag1'] >= 550 and row['level2_diag1'] < 554):
        df.loc[index, 'level2_diag1'] = 11
    elif (row['level2_diag1'] >= 555 and row['level2_diag1'] < 580):
        df.loc[index, 'level2_diag1'] = 12
    elif (np.floor(row['level2_diag1']) == 787):
        df.loc[index, 'level2_diag1'] = 13
    elif (np.floor(row['level2_diag1']) == 250):
        df.loc[index, 'level2_diag1'] = 14
    elif (row['level2_diag1'] >= 800 and row['level2_diag1'] < 1000):
        df.loc[index, 'level2_diag1'] = 15
    elif (row['level2_diag1'] >= 710 and row['level2_diag1'] < 740):
        df.loc[index, 'level2_diag1'] = 16
    elif (row['level2_diag1'] >= 580 and row['level2_diag1'] < 630):
        df.loc[index, 'level2_diag1'] = 17
    elif (np.floor(row['level2_diag1']) == 788):
        df.loc[index, 'level2_diag1'] = 18
    elif (row['level2_diag1'] >= 140 and row['level2_diag1'] < 240):
        df.loc[index, 'level2_diag1'] = 19
    elif row['level2_diag1'] >= 240 and row['level2_diag1'] < 280 and (np.floor(row['level2_diag1']) != 250):
        df.loc[index, 'level2_diag1'] = 20
    elif (row['level2_diag1'] >= 680 and row['level2_diag1'] < 710) or (np.floor(row['level2_diag1']) == 782):
        df.loc[index, 'level2_diag1'] = 21
    elif (row['level2_diag1'] >= 290 and row['level2_diag1'] < 320):
        df.loc[index, 'level2_diag1'] = 22
    else:
        df.loc[index, 'level2_diag1'] = 0
        
    if (row['level2_diag2'] >= 390 and row['level2_diag2'] < 399):
        df.loc[index, 'level2_diag2'] = 1
    elif (row['level2_diag2'] >= 401 and row['level2_diag2'] < 415):
        df.loc[index, 'level2_diag2'] = 2
    elif (row['level2_diag2'] >= 415 and row['level2_diag2'] < 460):
        df.loc[index, 'level2_diag2'] = 3
    elif (np.floor(row['level2_diag2']) == 785):
        df.loc[index, 'level2_diag2'] = 4
    elif (row['level2_diag2'] >= 460 and row['level2_diag2'] < 489):
        df.loc[index, 'level2_diag2'] = 5
    elif (row['level2_diag2'] >= 490 and row['level2_diag2'] < 497):
        df.loc[index, 'level2_diag2'] = 6
    elif (row['level2_diag2'] >= 500 and row['level2_diag2'] < 520):
        df.loc[index, 'level2_diag2'] = 7
    elif (np.floor(row['level2_diag2']) == 786):
        df.loc[index, 'level2_diag2'] = 8
    elif (row['level2_diag2'] >= 520 and row['level2_diag2'] < 530):
        df.loc[index, 'level2_diag2'] = 9
    elif (row['level2_diag2'] >= 530 and row['level2_diag2'] < 544):
        df.loc[index, 'level2_diag2'] = 10
    elif (row['level2_diag2'] >= 550 and row['level2_diag2'] < 554):
        df.loc[index, 'level2_diag2'] = 11
    elif (row['level2_diag2'] >= 555 and row['level2_diag2'] < 580):
        df.loc[index, 'level2_diag2'] = 12
    elif (np.floor(row['level2_diag2']) == 787):
        df.loc[index, 'level2_diag2'] = 13
    elif (np.floor(row['level2_diag2']) == 250):
        df.loc[index, 'level2_diag2'] = 14
    elif (row['level2_diag2'] >= 800 and row['level2_diag2'] < 1000):
        df.loc[index, 'level2_diag2'] = 15
    elif (row['level2_diag2'] >= 710 and row['level2_diag2'] < 740):
        df.loc[index, 'level2_diag2'] = 16
    elif (row['level2_diag2'] >= 580 and row['level2_diag2'] < 630):
        df.loc[index, 'level2_diag2'] = 17
    elif (np.floor(row['level2_diag2']) == 788):
        df.loc[index, 'level2_diag2'] = 18
    elif (row['level2_diag2'] >= 140 and row['level2_diag2'] < 240):
        df.loc[index, 'level2_diag2'] = 19
    elif row['level2_diag2'] >= 240 and row['level2_diag2'] < 280 and (np.floor(row['level2_diag2']) != 250):
        df.loc[index, 'level2_diag2'] = 20
    elif (row['level2_diag2'] >= 680 and row['level2_diag2'] < 710) or (np.floor(row['level2_diag2']) == 782):
        df.loc[index, 'level2_diag2'] = 21
    elif (row['level2_diag2'] >= 290 and row['level2_diag2'] < 320):
        df.loc[index, 'level2_diag2'] = 22
    else:
        df.loc[index, 'level2_diag2'] = 0
        
        
    if (row['level2_diag3'] >= 390 and row['level2_diag3'] < 399):
        df.loc[index, 'level2_diag3'] = 1
    elif (row['level2_diag3'] >= 401 and row['level2_diag3'] < 415):
        df.loc[index, 'level2_diag3'] = 2
    elif (row['level2_diag3'] >= 415 and row['level2_diag3'] < 460):
        df.loc[index, 'level2_diag3'] = 3
    elif (np.floor(row['level2_diag3']) == 785):
        df.loc[index, 'level2_diag3'] = 4
    elif (row['level2_diag3'] >= 460 and row['level2_diag3'] < 489):
        df.loc[index, 'level2_diag3'] = 5
    elif (row['level2_diag3'] >= 490 and row['level2_diag3'] < 497):
        df.loc[index, 'level2_diag3'] = 6
    elif (row['level2_diag3'] >= 500 and row['level2_diag3'] < 520):
        df.loc[index, 'level2_diag3'] = 7
    elif (np.floor(row['level2_diag3']) == 786):
        df.loc[index, 'level2_diag3'] = 8
    elif (row['level2_diag3'] >= 520 and row['level2_diag3'] < 530):
        df.loc[index, 'level2_diag3'] = 9
    elif (row['level2_diag3'] >= 530 and row['level2_diag3'] < 544):
        df.loc[index, 'level2_diag3'] = 10
    elif (row['level2_diag3'] >= 550 and row['level2_diag3'] < 554):
        df.loc[index, 'level2_diag3'] = 11
    elif (row['level2_diag3'] >= 555 and row['level2_diag3'] < 580):
        df.loc[index, 'level2_diag3'] = 12
    elif (np.floor(row['level2_diag3']) == 787):
        df.loc[index, 'level2_diag3'] = 13
    elif (np.floor(row['level2_diag3']) == 250):
        df.loc[index, 'level2_diag3'] = 14
    elif (row['level2_diag3'] >= 800 and row['level2_diag3'] < 1000):
        df.loc[index, 'level2_diag3'] = 15
    elif (row['level2_diag3'] >= 710 and row['level2_diag3'] < 740):
        df.loc[index, 'level2_diag3'] = 16
    elif (row['level2_diag3'] >= 580 and row['level2_diag3'] < 630):
        df.loc[index, 'level2_diag3'] = 17
    elif (np.floor(row['level2_diag3']) == 788):
        df.loc[index, 'level2_diag3'] = 18
    elif (row['level2_diag3'] >= 140 and row['level2_diag3'] < 240):
        df.loc[index, 'level2_diag3'] = 19
    elif row['level2_diag3'] >= 240 and row['level2_diag3'] < 280 and (np.floor(row['level2_diag3']) != 250):
        df.loc[index, 'level2_diag3'] = 20
    elif (row['level2_diag3'] >= 680 and row['level2_diag3'] < 710) or (np.floor(row['level2_diag3']) == 782):
        df.loc[index, 'level2_diag3'] = 21
    elif (row['level2_diag3'] >= 290 and row['level2_diag3'] < 320):
        df.loc[index, 'level2_diag3'] = 22
    else:
        df.loc[index, 'level2_diag3'] = 0

In [ ]:
df.shape

In [ ]:
df[['diag_1','level1_diag1','level2_diag1']].head(15).T

In [ ]:
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)
df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)
df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)

In [ ]:
# keys is the same as before
for col in keys:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)

In [ ]:
df['A1Cresult'] = df['A1Cresult'].replace('>7', 1)
df['A1Cresult'] = df['A1Cresult'].replace('>8', 1)
df['A1Cresult'] = df['A1Cresult'].replace('Norm', 0)
df['A1Cresult'] = df['A1Cresult'].replace('None', -99)
df['max_glu_serum'] = df['max_glu_serum'].replace('>200', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('>300', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('Norm', 0)
df['max_glu_serum'] = df['max_glu_serum'].replace('None', -99)

In [ ]:
age_dict = {'[0-10)':5, '[10-20)':15, '[20-30)':25, '[30-40)':35, '[40-50)':45, '[50-60)':55, '[60-70)':65, '[70-80)':75, '[80-90)':85, '[90-100)':95}
df['age'] = df.age.map(age_dict)
df['age'] = df['age'].astype('int64')
print(df.age.value_counts())

In [ ]:
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('NO', 0)

In [ ]:
df.to_csv('./diabetes_data_preprocessed_new.csv')

## (II) Modelling

In [3]:
df2 = pd.read_csv("diabetes_data_preprocessed_new.csv", index_col=0)
df2.shape
df2.head(10).T

,0,1,2,3,4,5,6,7,8,9
encounter_id,2278392,149190,64410,500364,16680,35754,55842,63768,12522,15738
patient_nbr,8222157,55629189,86047875,82442376,42519267,82637451,84259809,114882984,48330783,63555939
race,Caucasian,Caucasian,AfricanAmerican,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
gender,0,0,0,1,1,1,1,1,0,0
age,5,15,25,35,45,55,65,75,85,95
admission_type_id,5,1,1,1,1,1,3,1,1,3
discharge_disposition_id,18,1,1,1,1,1,1,1,1,2
admission_source_id,1,7,7,7,7,1,1,7,4,4
time_in_hospital,1,3,2,2,1,3,4,5,13,12
num_lab_procedures,41,59,11,44,51,31,70,73,68,33


In [4]:
# convert data type of nominal features in dataframe to 'object' type
to_cat = ['encounter_id', 'patient_nbr', 'gender', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',\
          'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', \
          'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose','miglitol', \
          'troglitazone', 'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', \
          'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', \
          'A1Cresult', 'max_glu_serum', 'level1_diag1', 'level1_diag2', 'level1_diag3', 'level2_diag1', 'level2_diag2', 'level2_diag3']

df2[to_cat] = df2[to_cat].astype('object')

In [5]:
df2.dtypes

encounter_id                object
patient_nbr                 object
race                        object
gender                      object
age                          int64
admission_type_id           object
discharge_disposition_id    object
admission_source_id         object
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide           

In [6]:
num_col = list(set(list(df2._get_numeric_data().columns))- {'readmitted'})
num_col

['time_in_hospital',
 'nummed',
 'number_emergency',
 'num_medications',
 'number_diagnoses',
 'service_utilization',
 'numchange',
 'num_procedures',
 'num_lab_procedures',
 'number_inpatient',
 'number_outpatient',
 'age']

In [7]:
# Removing skewnewss and kurtosis using log transformation if it is above a threshold value (2)

statdataframe = pd.DataFrame()
statdataframe['numeric_column'] = num_col
skew_before = []
skew_after = []

kurt_before = []
kurt_after = []

standard_deviation_before = []
standard_deviation_after = []

log_transform_needed = []

log_type = []

for i in num_col:
    skewval = df2[i].skew()
    skew_before.append(skewval)
    
    kurtval = df2[i].kurtosis()
    kurt_before.append(kurtval)
    
    sdval = df2[i].std()
    standard_deviation_before.append(sdval)
    
    if (abs(skewval) >2) & (abs(kurtval) >2):
        log_transform_needed.append('Yes')
        
        if len(df2[df2[i] == 0])/len(df2) <=0.02:
            log_type.append('log')
            skewvalnew = np.log(pd.DataFrame(df2[train_data[i] > 0])[i]).skew()
            skew_after.append(skewvalnew)
            
            kurtvalnew = np.log(pd.DataFrame(df2[train_data[i] > 0])[i]).kurtosis()
            kurt_after.append(kurtvalnew)
            
            sdvalnew = np.log(pd.DataFrame(df2[train_data[i] > 0])[i]).std()
            standard_deviation_after.append(sdvalnew)
            
        else:
            log_type.append('log1p')
            skewvalnew = np.log1p(pd.DataFrame(df2[df2[i] >= 0])[i]).skew()
            skew_after.append(skewvalnew)
        
            kurtvalnew = np.log1p(pd.DataFrame(df2[df2[i] >= 0])[i]).kurtosis()
            kurt_after.append(kurtvalnew)
            
            sdvalnew = np.log1p(pd.DataFrame(df2[df2[i] >= 0])[i]).std()
            standard_deviation_after.append(sdvalnew)
            
    else:
        log_type.append('NA')
        log_transform_needed.append('No')
        
        skew_after.append(skewval)
        kurt_after.append(kurtval)
        standard_deviation_after.append(sdval)

statdataframe['skew_before'] = skew_before
statdataframe['kurtosis_before'] = kurt_before
statdataframe['standard_deviation_before'] = standard_deviation_before
statdataframe['log_transform_needed'] = log_transform_needed
statdataframe['log_type'] = log_type
statdataframe['skew_after'] = skew_after
statdataframe['kurtosis_after'] = kurt_after
statdataframe['standard_deviation_after'] = standard_deviation_after

In [8]:
statdataframe

,numeric_column,skew_before,kurtosis_before,standard_deviation_before,log_transform_needed,log_type,skew_after,kurtosis_after,standard_deviation_after
0,time_in_hospital,1.137931,0.871189,2.974528,No,NA,1.137931,0.871189,2.974528
1,nummed,0.675116,0.277244,0.921620,No,NA,0.675116,0.277244,0.921620
2,number_emergency,22.842251,1185.246374,0.935517,Yes,log1p,3.661064,16.276610,0.315405
3,num_medications,1.333039,3.523472,8.092612,No,NA,1.333039,3.523472,8.092612
4,number_diagnoses,-0.867785,-0.109005,1.938211,No,NA,-0.867785,-0.109005,1.938211
5,service_utilization,5.334724,67.779770,2.292730,Yes,log1p,1.119136,0.547353,0.662373
6,numchange,1.425310,1.433619,0.487858,No,NA,1.425310,1.433619,0.487858
7,num_procedures,1.326020,0.890773,1.700335,No,NA,1.326020,0.890773,1.700335
8,num_lab_procedures,-0.241491,-0.253099,19.620228,No,NA,-0.241491,-0.253099,19.620228
9,number_inpatient,3.626402,20.833542,1.261825,Yes,log1p,1.450492,1.405364,0.510433


In [9]:
# performing the log transformation for the columns determined to be needing it above.

for i in range(len(statdataframe)):
    if statdataframe['log_transform_needed'][i] == 'Yes':
        colname = str(statdataframe['numeric_column'][i])
        
        if statdataframe['log_type'][i] == 'log':
            df2 = df2[df2[colname] > 0]
            df2[colname + "_log"] = np.log(df2[colname])
            
        elif statdataframe['log_type'][i] == 'log1p':
            df2 = df2[df2[colname] >= 0]
            df2[colname + "_log1p"] = np.log1p(df2[colname])

In [10]:
df2 = df2.drop(['number_outpatient', 'number_inpatient', 'number_emergency','service_utilization'], axis = 1)

In [11]:
df2.shape

(100120, 54)

In [12]:
df2.head(10).T

,0,1,2,3,4,5,6,7,8,9
encounter_id,2278392,149190,64410,500364,16680,35754,55842,63768,12522,15738
patient_nbr,8222157,55629189,86047875,82442376,42519267,82637451,84259809,114882984,48330783,63555939
race,Caucasian,Caucasian,AfricanAmerican,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
gender,0,0,0,1,1,1,1,1,0,0
age,5,15,25,35,45,55,65,75,85,95
admission_type_id,5,1,1,1,1,1,3,1,1,3
discharge_disposition_id,18,1,1,1,1,1,1,1,1,2
admission_source_id,1,7,7,7,7,1,1,7,4,4
time_in_hospital,1,3,2,2,1,3,4,5,13,12
num_lab_procedures,41,59,11,44,51,31,70,73,68,33


In [13]:
numerics = list(set(list(df2._get_numeric_data().columns))- {'readmitted'})
numerics

['time_in_hospital',
 'nummed',
 'num_medications',
 'number_diagnoses',
 'number_emergency_log1p',
 'number_outpatient_log1p',
 'numchange',
 'num_procedures',
 'num_lab_procedures',
 'service_utilization_log1p',
 'number_inpatient_log1p',
 'age']

In [14]:
df2.dtypes

encounter_id                  object
patient_nbr                   object
race                          object
gender                        object
age                            int64
admission_type_id             object
discharge_disposition_id      object
admission_source_id           object
time_in_hospital               int64
num_lab_procedures             int64
num_procedures                 int64
num_medications                int64
diag_1                        object
diag_2                        object
diag_3                        object
number_diagnoses               int64
max_glu_serum                 object
A1Cresult                     object
metformin                     object
repaglinide                   object
nateglinide                   object
chlorpropamide                object
glimepiride                   object
acetohexamide                 object
glipizide                     object
glyburide                     object
tolbutamide                   object
p

In [15]:
df2.shape

(100120, 54)

In [16]:
#interactionterms = [('num_medications','time_in_hospital'),
#('num_medications','num_procedures'),
#('time_in_hospital','num_lab_procedures'),
#('num_medications','num_lab_procedures'),
#('num_medications','number_diagnoses'),
#('age','number_diagnoses'),
#('change','num_medications'),
#('number_diagnoses','time_in_hospital'),
#('num_medications','numchange')]

In [17]:
#for inter in interactionterms:
#    name = inter[0] + '|' + inter[1]
#    df2[name] = df2[inter[0]] * df2[inter[1]]

In [18]:
datf = pd.DataFrame()
datf['features'] = numerics
datf['std_dev'] = datf['features'].apply(lambda x: df2[x].std())
datf['mean'] = datf['features'].apply(lambda x: df2[x].mean())

In [19]:
datf

,features,std_dev,mean
0,time_in_hospital,2.974528,4.389453
1,nummed,0.921620,1.185537
2,num_medications,8.092612,15.981552
3,number_diagnoses,1.938211,7.409339
4,number_emergency_log1p,0.315405,0.101963
5,number_outpatient_log1p,0.429394,0.172340
6,numchange,0.487858,0.287545
7,num_procedures,1.700335,1.330803
8,num_lab_procedures,19.620228,42.943787
9,service_utilization_log1p,0.662373,0.513654


In [20]:
df2 = df2.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
df2.shape

(70442, 54)

In [21]:
df2[numerics].dtypes

time_in_hospital               int64
nummed                         int64
num_medications                int64
number_diagnoses               int64
number_emergency_log1p       float64
number_outpatient_log1p      float64
numchange                      int64
num_procedures                 int64
num_lab_procedures             int64
service_utilization_log1p    float64
number_inpatient_log1p       float64
age                            int64
dtype: object

In [22]:
def standardize(raw_data):
    return ((raw_data - np.mean(raw_data, axis = 0)) / np.std(raw_data, axis = 0))
df2[numerics] = standardize(df2[numerics])
import scipy as sp
import scipy.stats
df2[(np.abs(sp.stats.zscore(df2[numerics])) < 3).all(axis=1)]

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,level1_diag1,level2_diag1,level1_diag2,level2_diag2,level1_diag3,level2_diag3,number_emergency_log1p,service_utilization_log1p,number_inpatient_log1p,number_outpatient_log1p
2,64410,86047875,AfricanAmerican,0,-2.536028,1,1,7,-0.776736,-1.603864,...,0,0,4,14,0,0,-0.264072,2.218262,1.995771,2.538861
3,500364,82442376,Caucasian,1,-1.910135,1,1,7,-0.776736,0.054368,...,0,0,4,14,1,2,-0.264072,-0.532287,-0.341645,-0.352722
4,16680,42519267,Caucasian,1,-1.284243,1,1,7,-1.116914,0.406114,...,8,19,8,19,4,14,-0.264072,-0.532287,-0.341645,-0.352722
5,35754,82637451,Caucasian,1,-0.658350,1,1,1,-0.436559,-0.598875,...,1,2,1,2,4,14,-0.264072,-0.532287,-0.341645,-0.352722
6,55842,84259809,Caucasian,1,-0.032458,3,1,1,-0.096381,1.360853,...,1,2,1,2,0,0,-0.264072,-0.532287,-0.341645,-0.352722
7,63768,114882984,Caucasian,1,0.593435,1,1,7,0.243797,1.511602,...,1,3,2,6,4,14,-0.264072,-0.532287,-0.341645,-0.352722
8,12522,48330783,Caucasian,0,1.219327,1,1,4,2.965218,1.260354,...,1,1,1,3,0,0,-0.264072,-0.532287,-0.341645,-0.352722
9,15738,63555939,Caucasian,0,1.845220,3,2,4,2.625041,-0.498376,...,1,3,8,19,2,5,-0.264072,-0.532287,-0.341645,-0.352722
10,28236,89869032,AfricanAmerican,0,-1.284243,1,1,7,1.604508,0.205116,...,4,14,1,2,5,15,-0.264072,-0.532287,-0.341645,-0.352722
11,36900,77391171,AfricanAmerican,1,-0.032458,1,1,4,0.924152,0.958858,...,8,19,0,0,8,19,-0.264072,-0.532287,-0.341645,-0.352722


In [23]:
import seaborn as sns
from matplotlib.colors import ListedColormap
my_cmap = ListedColormap(sns.light_palette((250, 100, 50), input="husl", n_colors=50).as_hex())
df2 = df2.drop(['patient_nbr', 'encounter_id'], axis=1)

table.style.background_gradient(cmap=my_cmap, axis = 0)

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,readmitted,numchange,nummed,number_emergency_log1p,service_utilization_log1p,number_inpatient_log1p,number_outpatient_log1p
age,1,0.128561,0.0266174,-0.02434,0.0544389,0.254252,0.0457358,-0.067106,-0.00361263,-0.0443524,0.0237037,0.0262302,0.032686
time_in_hospital,0.128561,1,0.332129,0.187856,0.467931,0.23437,0.0546615,0.16797,0.085771,-0.00759536,0.0169746,0.0704713,-0.021992
num_lab_procedures,0.0266174,0.332129,1,0.0437017,0.259065,0.154574,0.0317229,0.12901,0.0428562,0.017373,0.0303425,0.0849385,-0.0196605
num_procedures,-0.02434,0.187856,0.0437017,1,0.400298,0.0855402,7.41584e-05,0.0187962,0.00506419,-0.0404431,-0.0442299,-0.023825,-0.0228547
num_medications,0.0544389,0.467931,0.259065,0.400298,1,0.2586,0.0360049,0.2273,0.237304,0.00835637,0.045897,0.0429493,0.0328306
number_diagnoses,0.254252,0.23437,0.154574,0.0855402,0.2586,1,0.0443419,0.0654594,0.0178828,0.0648761,0.123683,0.0793302,0.0903418
readmitted,0.0457358,0.0546615,0.0317229,7.41584e-05,0.0360049,0.0443419,1,0.0226039,0.0117976,0.0330646,0.0684923,0.09678,0.0138156
numchange,-0.067106,0.16797,0.12901,0.0187962,0.2273,0.0654594,0.0226039,1,0.316755,0.0421867,0.0358661,0.0108163,0.0180822
nummed,-0.00361263,0.085771,0.0428562,0.00506419,0.237304,0.0178828,0.0117976,0.316755,1,0.0190974,0.0203282,-0.00469878,0.0205915
number_emergency_log1p,-0.0443524,-0.00759536,0.017373,-0.0404431,0.00835637,0.0648761,0.0330646,0.0421867,0.0190974,1,0.505888,0.157071,0.140815


In [24]:
pd.options.display.max_rows = 400

c = df2.corr().abs()
s = c.unstack()
print(s.shape)
so = s.sort_values(ascending=False)

so[10:120]

(169,)


service_utilization_log1p  service_utilization_log1p    1.000000
readmitted                 readmitted                   1.000000
age                        age                          1.000000
service_utilization_log1p  number_outpatient_log1p      0.753627
number_outpatient_log1p    service_utilization_log1p    0.753627
number_inpatient_log1p     service_utilization_log1p    0.605494
service_utilization_log1p  number_inpatient_log1p       0.605494
number_emergency_log1p     service_utilization_log1p    0.505888
service_utilization_log1p  number_emergency_log1p       0.505888
num_medications            time_in_hospital             0.467931
time_in_hospital           num_medications              0.467931
num_medications            num_procedures               0.400298
num_procedures             num_medications              0.400298
num_lab_procedures         time_in_hospital             0.332129
time_in_hospital           num_lab_procedures           0.332129
numchange                

In [25]:
df2.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'numchange', 'nummed', 'level1_diag1', 'level2_diag1', 'level1_diag2',
       'level2_diag2', 'level1_diag3', 'level2_diag3',
       'number_emergency_log1p', 'service_utilization_log1p',
       'number_inpatient_log1p', 'number_outpatient_log1p'],
      dtype='object'

In [26]:
df2.head(10).T

,0,1,2,3,4,5,6,7,8,9
race,Caucasian,Caucasian,AfricanAmerican,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
gender,0,0,0,1,1,1,1,1,0,0
age,-3.78781,-3.16192,-2.53603,-1.91014,-1.28424,-0.65835,-0.0324577,0.593435,1.21933,1.84522
admission_type_id,5,1,1,1,1,1,3,1,1,3
discharge_disposition_id,18,1,1,1,1,1,1,1,1,2
admission_source_id,1,7,7,7,7,1,1,7,4,4
time_in_hospital,-1.11691,-0.436559,-0.776736,-0.776736,-1.11691,-0.436559,-0.0963809,0.243797,2.96522,2.62504
num_lab_procedures,-0.0963808,0.808109,-1.60386,0.0543676,0.406114,-0.598875,1.36085,1.5116,1.26035,-0.498376
num_procedures,-0.810421,-0.810421,2.03869,-0.240598,-0.810421,2.60851,-0.240598,-0.810421,0.329224,0.899046
num_medications,-1.77072,0.28178,-0.321897,0.0403091,-0.925574,0.0403091,0.643986,-0.442632,1.48913,0.28178


In [44]:
pd.DataFrame([df2.level1_diag1.astype(int),df2.level2_diag1.astype(int)]).T.corr(method='pearson')

,level1_diag1,level2_diag1
level1_diag1,1.000000,0.639884
level2_diag1,0.639884,1.000000


In [45]:
df2.dtypes

race                          object
gender                        object
age                          float64
admission_type_id             object
discharge_disposition_id      object
admission_source_id           object
time_in_hospital             float64
num_lab_procedures           float64
num_procedures               float64
num_medications              float64
diag_1                        object
diag_2                        object
diag_3                        object
number_diagnoses             float64
max_glu_serum                 object
A1Cresult                     object
metformin                     object
repaglinide                   object
nateglinide                   object
chlorpropamide                object
glimepiride                   object
acetohexamide                 object
glipizide                     object
glyburide                     object
tolbutamide                   object
pioglitazone                  object
rosiglitazone                 object
a

In [47]:
df2.to_csv('./diabetes_data_preprocessed_new_df2.csv')

In [ ]:
df2.drop(['diag_1', 'diag_2', 'diag_3', 'level2_diag1', 'level2_diag2',
         'level2_diag3'], axis=1, inplace=True)

In [56]:
df3 = df2

In [66]:
df_pd = pd.get_dummies(df3, drop_first = True)

In [80]:
len(df_pd.columns)

85

In [82]:
len(list(set(df_pd.columns)-{'readmitted'}))

84

In [84]:
X = df_pd[list(set(df_pd.columns)-{'readmitted'})]
y = df_pd['readmitted']

In [98]:
X.shape

(70442, 84)

In [99]:
y.shape

(70442,)

## Models - Part I - Regular split, no resampling

### Logistic Regression - Regular

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
X_train, X_dev, Y_train, Y_dev = train_test_split(X, y, test_size=0.20, random_state=0)
logreg = LogisticRegression(fit_intercept=True, penalty='l1')
print("Cross Validation Score: {:.2%}".format(np.mean(cross_val_score(logreg, X_train, Y_train, cv=10))))
logreg.fit(X_train, Y_train)
print("Dev Set score: {:.2%}".format(logreg.score(X_dev, Y_dev)))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

Cross Validation Score: 91.04%


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Dev Set score: 91.04%


In [87]:
Y_dev_predict = logreg.predict(X_dev)

In [97]:
np.bincount(Y_dev_predict)

array([14083,     6])

In [88]:
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,All
Actual,,
0,1951,1951
1,214,214
All,2165,2165


In [128]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy  is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall    is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("F1_score  is {0:.2f}".format(f1_score(Y_dev, Y_dev_predict)))

Accuracy  is 0.91
Precision is 0.00
Recall    is 0.00
F1_score  is 0.00


### Decision Tree(entropy) - Regular

In [130]:
from sklearn.tree import DecisionTreeClassifier
dte = DecisionTreeClassifier(max_depth=28, criterion = "entropy", min_samples_split=10)
print("Cross Validation score: {:.2%}".format(np.mean(cross_val_score(dte, X_train, Y_train, cv=10))))
dte.fit(X_train, Y_train)
print("Dev Set score: {:.2%}".format(dte.score(X_dev, Y_dev)))

Cross Validation score: 86.28%
Dev Set score: 85.72%


In [131]:
Y_dev_predict = dte.predict(X_dev)

In [132]:
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,1,All
Actual,,,
0,1830,121,1951
1,195,19,214
All,2025,140,2165


In [134]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
print("Accuracy is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("AUC is {0:.2f}".format(roc_auc_score(Y_dev, Y_dev_predict)))
print('F1_score is {0:.2f}'.format(f1_score(Y_dev, Y_dev_predict)))

#accuracy_dte = accuracy_score(Y_dev, Y_dev_predict)
#precision_dte = precision_score(Y_dev, Y_dev_predict)
#recall_dte = recall_score(Y_dev, Y_dev_predict)
#auc_dte = roc_auc_score(Y_dev, Y_dev_predict)

Accuracy is 0.86
Precision is 0.12
Recall is 0.10
AUC is 0.51
F1_score is 0.11


### Decision Tree(gini) - Regular

In [136]:
from sklearn.tree import DecisionTreeClassifier
dtg = DecisionTreeClassifier(max_depth=28, criterion = "gini", min_samples_split=10)
print("Cross Validation score: {:.2%}".format(np.mean(cross_val_score(dte, X_train, Y_train, cv=10))))
dtg.fit(X_train, Y_train)
print("Dev Set score: {:.2%}".format(dtg.score(X_dev, Y_dev)))

Cross Validation score: 86.70%
Dev Set score: 86.42%


In [137]:
Y_dev_predict = dtg.predict(X_dev)

In [138]:
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,1,All
Actual,,,
0,1850,101,1951
1,197,17,214
All,2047,118,2165


In [140]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("AUC is {0:.2f}".format(roc_auc_score(Y_dev, Y_dev_predict)))
print('F1_score is {0:.2f}'.format(f1_score(Y_dev, Y_dev_predict)))

Accuracy is 0.86
Precision is 0.13
Recall is 0.09
AUC is 0.51
F1_score is 0.10


### Random Forest(Entropy) - Regular

In [144]:
from sklearn.ensemble import RandomForestClassifier
forrest = RandomForestClassifier(n_estimators = 10, max_depth=25, criterion = "entropy", min_samples_split=10)
print("Cross Validation score: {:.2%}".format(np.mean(cross_val_score(forrest, X_train, Y_train, cv=10))))
forrest.fit(X_train, Y_train)
print("Dev Set score: {:.2%}".format(forrest.score(X_dev, Y_dev)))

Cross Validation score: 91.04%
Dev Set score: 91.03%


In [145]:
Y_dev_predict = forrest.predict(X_dev)
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,1,All
Actual,,,
0,1950,1,1951
1,212,2,214
All,2162,3,2165


In [146]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("AUC is {0:.2f}".format(roc_auc_score(Y_dev, Y_dev_predict)))
print("F1_score is {0:.2f}".format(f1_score(Y_dev, Y_dev_predict)))

Accuracy is 0.91
Precision is 0.18
Recall is 0.00
AUC is 0.50
F1_score is 0.00


### Random Forest (gini) - Regular

In [147]:
from sklearn.ensemble import RandomForestClassifier
forrest = RandomForestClassifier(n_estimators = 10, max_depth=25, criterion = "gini", min_samples_split=10)
print("Cross Validation score: {:.2%}".format(np.mean(cross_val_score(forrest, X_train, Y_train, cv=10))))
forrest.fit(X_train, Y_train)
print("Dev Set score: {:.2%}".format(forrest.score(X_dev, Y_dev)))

Cross Validation score: 91.00%
Dev Set score: 90.99%


In [148]:
Y_dev_predict = forrest.predict(X_dev)
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,1,All
Actual,,,
0,1949,2,1951
1,214,0,214
All,2163,2,2165


In [149]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("AUC is {0:.2f}".format(roc_auc_score(Y_dev, Y_dev_predict)))
print("F1_score is {0:.2f}".format(f1_score(Y_dev, Y_dev_predict)))

Accuracy is 0.91
Precision is 0.12
Recall is 0.00
AUC is 0.50
F1_score is 0.00


## Models - Part II - Split first and SMOTE over-sampling on training set

In [159]:
# Data balancing applied using SMOTE
from imblearn.over_sampling import SMOTE
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y_train)))
sm = SMOTE(random_state=20)
X_train_new, Y_train_new = sm.fit_sample(X_train, Y_train)
print('New dataset shape {}'.format(Counter(Y_train_new)))

Original dataset shape Counter({0: 51314, 1: 5039})
New dataset shape Counter({0: 51314, 1: 51314})


### Decesion Tree(Entropy) - After SMOTE

In [163]:
from sklearn.tree import DecisionTreeClassifier
dte = DecisionTreeClassifier(max_depth=28, criterion = "entropy", min_samples_split=10)
print("Cross Validation score: {:.2%}".format(np.mean(cross_val_score(dte, X_train_new, Y_train_new, cv=10))))
dte.fit(X_train_new, Y_train_new)
print("Dev Set score: {:.2%}".format(dte.score(X_dev, Y_dev)))

Cross Validation score: 91.06%
Dev Set score: 85.41%


In [168]:
Y_dev_predict = dte.predict(X_dev)
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,1,All
Actual,,,
0,1838,113,1951
1,197,17,214
All,2035,130,2165


In [169]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("AUC is {0:.2f}".format(roc_auc_score(Y_dev, Y_dev_predict)))
print("F1_score is {0:.2f}".format(f1_score(Y_dev, Y_dev_predict)))

Accuracy is 0.85
Precision is 0.10
Recall is 0.08
AUC is 0.50
F1_score is 0.08


### Random Forest(Entropy) - After SMOTE

In [164]:
from sklearn.ensemble import RandomForestClassifier
forrest = RandomForestClassifier(n_estimators = 10, max_depth=25, criterion = "entropy", min_samples_split=10)
print("Cross Validation score: {:.2%}".format(np.mean(cross_val_score(forrest, X_train_new, Y_train_new, cv=10))))
forrest.fit(X_train_new, Y_train_new)
print("Dev Set score: {:.2%}".format(forrest.score(X_dev, Y_dev)))

Cross Validation score: 94.60%
Dev Set score: 90.77%


In [171]:
Y_dev_predict = forrest.predict(X_dev)
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,1,All
Actual,,,
0,1940,11,1951
1,212,2,214
All,2152,13,2165


In [172]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("AUC is {0:.2f}".format(roc_auc_score(Y_dev, Y_dev_predict)))
print("F1_score is {0:.2f}".format(f1_score(Y_dev, Y_dev_predict)))

Accuracy is 0.91
Precision is 0.23
Recall is 0.02
AUC is 0.51
F1_score is 0.03


## Models - Part III - SMOTE over-sampling on whole dataset and then split train and dev

In [174]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))
sm = SMOTE(random_state=20)
X_new, y_new = sm.fit_sample(X, y)
print('New dataset shape {}'.format(Counter(y_new)))

Original dataset shape Counter({0: 64146, 1: 6296})
New dataset shape Counter({0: 64146, 1: 64146})


In [183]:
X_train, X_dev, Y_train, Y_dev = train_test_split(X_new, y_new, test_size=0.20, random_state=0)

### Random Forest(Entropy) - SMOTE first and then split

In [184]:
from sklearn.ensemble import RandomForestClassifier
forrest = RandomForestClassifier(n_estimators = 10, max_depth=25, criterion = "entropy", min_samples_split=10)
print("Cross Validation score: {:.2%}".format(np.mean(cross_val_score(forrest, X_train, Y_train, cv=10))))
forrest.fit(X_train, Y_train)
print("Dev Set score: {:.2%}".format(forrest.score(X_dev, Y_dev)))

Cross Validation score: 94.63%
Dev Set score: 94.73%


In [185]:
Y_dev_predict = forrest.predict(X_dev)
pd.crosstab(pd.Series(Y_dev, name = 'Actual'), pd.Series(Y_dev_predict, name = 'Predict'), margins = True)

Predict,0,1,All
Actual,,,
0,12672,67,12739
1,1285,11635,12920
All,13957,11702,25659


In [186]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy is {0:.2f}".format(accuracy_score(Y_dev, Y_dev_predict)))
print("Precision is {0:.2f}".format(precision_score(Y_dev, Y_dev_predict)))
print("Recall is {0:.2f}".format(recall_score(Y_dev, Y_dev_predict)))
print("AUC is {0:.2f}".format(roc_auc_score(Y_dev, Y_dev_predict)))
print("F1_score is {0:.2f}".format(f1_score(Y_dev, Y_dev_predict)))

Accuracy is 0.95
Precision is 0.99
Recall is 0.90
AUC is 0.95
F1_score is 0.95
